# Introduction to Bedrock - Continual Pre-Training

> *If you see errors, you may need to be allow-listed for the Bedrock models used by this notebook*

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*


In this demo notebook, we demonstrate how to use the Bedrock Python SDK for fine-tuning Bedrock models with your own data. If you have text samples to train and want to adapt the Bedrock models to your domain, you can further fine-tune the Bedrock foundation models by providing your own training datasets. You can upload your datasets to Amazon S3, and provide the S3 bucket path while configuring a Bedrock fine-tuning job. You can also adjust hyper parameters (learning rate, epoch, and batch size) for fine-tuning. After the fine-tuning job of the model with your dataset has completed, you can start using the model for inference in the Bedrock playground application. You can select the fine-tuned model and submit a prompt to the fine-tuned model along with a set of model parameters. The fine-tuned model should generate texts to be more alike your text samples. 

-----------

1. Setup
2. Fine-tuning
3. Testing the fine-tuned model

 Note: This notebook was tested in Amazon SageMaker Studio with Python 3 (Data Science 2.0) kernel.

---

## 1. Setup

In [2]:
%pip install -U --force-reinstall pandas==2.1.2

  Obtaining dependency information for pandas==2.1.2 from https://files.pythonhosted.org/packages/02/52/815f643ed3afb3365354548b3c8b557dbf926a65c40ad5b6d9e455147c7e/pandas-2.1.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached pandas-2.1.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (18 kB)
  Obtaining dependency information for numpy<2,>=1.22.4 from https://files.pythonhosted.org/packages/2d/5e/cb38e3d1916cc29880c84a9332a9122a8f49a7b57ec7aea63e0f678587a2/numpy-1.26.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached numpy-1.26.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached python_dateutil-2.8.2-py2.py3-none-any.whl (247 kB)
  Obtaining dependency information for pytz>=2020.1 from https://files.pythonhosted.org/packages/32/4d/aaf7eff5deb402fd9a24a1449a8119f00d74ae9c2efa79f8ef9994261fc2/pytz-2023.3.post1-py2.py3-none-any.whl.metadata
  Using cached pyt

In [3]:
%pip install -U --force-reinstall \
             langchain==0.0.324 \
             typing_extensions==4.7.1 \
             pypdf==3.16.4

  Obtaining dependency information for langchain==0.0.324 from https://files.pythonhosted.org/packages/b0/6c/85e5ee99dca8a960d89a7dab2a5cc02c3a30f12ad2987782842976064e16/langchain-0.0.324-py3-none-any.whl.metadata
  Using cached langchain-0.0.324-py3-none-any.whl.metadata (15 kB)
  Obtaining dependency information for typing_extensions==4.7.1 from https://files.pythonhosted.org/packages/ec/6b/63cc3df74987c36fe26157ee12e09e8f9db4de771e0f3404263117e75b95/typing_extensions-4.7.1-py3-none-any.whl.metadata
  Using cached typing_extensions-4.7.1-py3-none-any.whl.metadata (3.1 kB)
  Obtaining dependency information for pypdf==3.16.4 from https://files.pythonhosted.org/packages/89/b4/c751015b8802bcd4f7705580ac5e84f01f1d09ca38f1cff6bf1ad680bc43/pypdf-3.16.4-py3-none-any.whl.metadata
  Using cached pypdf-3.16.4-py3-none-any.whl.metadata (7.4 kB)
  Obtaining dependency information for PyYAML>=5.3 from https://files.pythonhosted.org/packages/29/61/bf33c6c85c55bc45a29eee3195848ff2d518d84735eb0e2d8c

# YOU MUST RESTART KERNEL AFTER pip install typing_extensions ^^ ABOVE ^^

In [4]:
%pip install -U boto3-1.28.60-py3-none-any.whl \
               botocore-1.31.60-py3-none-any.whl \
               awscli-1.29.60-py3-none-any.whl \
               --force-reinstall --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.3.3 requires pyqt5<5.16, which is not installed.
spyder 5.3.3 requires pyqtwebengine<5.16, which is not installed.
distributed 2022.7.0 requires tornado<6.2,>=6.0.3, but you have tornado 6.3.3 which is incompatible.
jupyterlab 3.4.4 requires jupyter-server~=1.16, but you have jupyter-server 2.7.3 which is incompatible.
jupyterlab-server 2.10.3 requires jupyter-server~=1.4, but you have jupyter-server 2.7.3 which is incompatible.
notebook 6.5.6 requires jupyter-client<8,>=5.3.4, but you have jupyter-client 8.4.0 which is incompatible.
notebook 6.5.6 requires pyzmq<25,>=17, but you have pyzmq 25.1.1 which is incompatible.
panel 0.13.1 requires bokeh<2.5.0,>=2.4.0, but you have bokeh 3.3.0 which is incompatible.
pyasn1-modules 0.2.8 requires pyasn1<0.5.0,>=0.4.6, but you have pyasn1 0.5.0 which is incompatib

In [5]:
%pip list | grep boto3

boto3                                1.28.60

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


#### Now let's set up our connection to the Amazon Bedrock SDK using Boto3

In [6]:
#### Un comment the following lines to run from your local environment outside of the AWS account with Bedrock access

#import os
#os.environ['BEDROCK_ASSUME_ROLE'] = '<YOUR_VALUES>'
#os.environ['AWS_PROFILE'] = '<YOUR_VALUES>'

In [7]:
import boto3
import json
import os
import sys

from utils import bedrock, print_ww

bedrock_admin = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
    runtime=False  # Needed for control plane
)

bedrock_runtime = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
    runtime=True  # Needed for control plane
)

Create new client
  Using region: us-west-2
boto3 Bedrock client successfully created!
bedrock(https://bedrock.us-west-2.amazonaws.com)
Create new client
  Using region: us-west-2
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-west-2.amazonaws.com)


In [8]:
bedrock_admin.list_foundation_models()

{'ResponseMetadata': {'RequestId': 'ce920ffb-26d9-417a-b09d-cac21e3e81a5',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 02 Nov 2023 02:47:52 GMT',
   'content-type': 'application/json',
   'content-length': '6095',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'ce920ffb-26d9-417a-b09d-cac21e3e81a5'},
  'RetryAttempts': 0},
 'modelSummaries': [{'modelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-tg1-large',
   'modelId': 'amazon.titan-tg1-large',
   'modelName': 'Titan Text Large',
   'providerName': 'Amazon',
   'inputModalities': ['TEXT'],
   'outputModalities': ['TEXT'],
   'responseStreamingSupported': True,
   'customizationsSupported': ['FINE_TUNING'],
   'inferenceTypesSupported': ['ON_DEMAND']},
  {'modelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-e1t-medium',
   'modelId': 'amazon.titan-e1t-medium',
   'modelName': 'Titan Text Embeddings',
   'providerName': 'Amazon',
   'inputModalities': ['TEXT'],
   'outputModalities'

In [9]:
import sagemaker

sess = sagemaker.Session()
sagemaker_session_bucket = sess.default_bucket()
role = sagemaker.get_execution_role()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


### Invoke Model before Continued Pre-Training

In [10]:
base_model_id = "amazon.titan-text-express-v1"
#base_model_id = "amazon.titan-text-lite-v1"

In [11]:
response = bedrock_runtime.invoke_model(
    # modelId needs to be Provisioned Throughput Model ARN
    modelId=base_model_id,
    body="""
{
  "inputText": "Where is AWS investing heavily in 2022?",
  "textGenerationConfig":{
    "maxTokenCount": 50, 
    "stopSequences": [],
    "temperature": 0.1,
    "topP": 0.9
  }
}
"""
)

response_body = response["body"].read().decode('utf8')
print(response_body)

print(json.loads(response_body)["results"][0]["outputText"])

{"inputTextTokenCount":12,"results":[{"tokenCount":39,"outputText":"\nAWS is investing heavily in the following areas in 2022:\n1. Quantum Computing\n2. AI and ML\n3. Databases\n4. Compute and Storage","completionReason":"FINISH"}]}

AWS is investing heavily in the following areas in 2022:
1. Quantum Computing
2. AI and ML
3. Databases
4. Compute and Storage


### Convert PDF to jsonlines format

In [12]:
from urllib.request import urlretrieve
urls = [
    'https://s2.q4cdn.com/299287126/files/doc_financials/2023/ar/2022-Shareholder-Letter.pdf',
]

filenames = [
    'AMZN-2022-Shareholder-Letter.pdf',
]

metadata = [
    dict(year=2022, source=filenames[0])
]

data_root = "./"

for idx, url in enumerate(urls):
    file_path = data_root + filenames[idx]
    urlretrieve(url, file_path)

In [13]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("./AMZN-2022-Shareholder-Letter.pdf")
document = loader.load()

# - in our testing Character split works better with this PDF data set
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 20000, # 4096 tokens * 6 chars per token = 24,576 
    chunk_overlap = 2000, # overlap for continuity across chunks
)

docs = text_splitter.split_documents(document)
docs[0]

Document(page_content='Dear shareholders:\nAs I sit down to write my second annual shareholder letter as CEO, I find myself optimistic and energized\nby what lies ahead for Amazon. Despite 2022 being one of the harder macroeconomic years in recent memory,and with some of our own operating challenges to boot, we still found a way to grow demand (on top ofthe unprecedented growth we experienced in the first half of the pandemic). We innovated in our largestbusinesses to meaningfully improve customer experience short and long term. And, we made importantadjustments in our investment decisions and the way in which we’ll invent moving forward, while stillpreserving the long-term investments that we believe can change the future of Amazon for customers,\nshareholders, and employees.\nWhile there were an unusual number of simultaneous challenges this past year, the reality is that if you\noperate in large, dynamic, global market segments with many capable and well-funded competitors (thecondi

In [14]:
import json

contents = ""
for doc in docs:
    content = {"input": doc.page_content}
    contents += (json.dumps(content) + "\n")
    
print(contents[:1000])

{"input": "Dear shareholders:\nAs I sit down to write my second annual shareholder letter as CEO, I find myself optimistic and energized\nby what lies ahead for Amazon. Despite 2022 being one of the harder macroeconomic years in recent memory,and with some of our own operating challenges to boot, we still found a way to grow demand (on top ofthe unprecedented growth we experienced in the first half of the pandemic). We innovated in our largestbusinesses to meaningfully improve customer experience short and long term. And, we made importantadjustments in our investment decisions and the way in which we\u2019ll invent moving forward, while stillpreserving the long-term investments that we believe can change the future of Amazon for customers,\nshareholders, and employees.\nWhile there were an unusual number of simultaneous challenges this past year, the reality is that if you\noperate in large, dynamic, global market segments with many capable and well-funded competitors (theconditions i

In [15]:
with open("./train-continual-pretraining.jsonl", "w") as file:
    file.writelines(contents)
    file.close()

In [16]:
import pandas as pd
df = pd.read_json("./train-continual-pretraining.jsonl", lines=True)
df

,input
0,Dear shareholders:\nAs I sit down to write my ...
1,"Over the last several months, we took a deep l..."
2,work came from optimizing the connections betw...
3,of the Amazon shopping experience for more tha...
4,"Beyond geographic expansion, we’ve been workin..."
5,"month flat fee, enables Prime members to get a..."
6,developer productivity by generating code sugg...
7,1997 LETTER TO SHAREHOLDERS\n(Reprinted from t...
8,• We will make bold rather than timid investme...
9,"Infrastructure\nDuring 1997, we worked hard to..."


In [17]:
data = "./train-continual-pretraining.jsonl"

Read the JSON line file into an object like any normal file

In [18]:
with open(data) as f:
    lines = f.read().splitlines()

#### Load the ‘lines’ object into a pandas Data Frame.

In [19]:
import pandas as pd
df_inter = pd.DataFrame(lines)
df_inter.columns = ['json_element']

This intermediate data frame will have only one column with each json object in a row. A sample output is given below.

In [20]:
df_inter['json_element'].apply(json.loads)

0    {'input': 'Dear shareholders:
As I sit down to...
1    {'input': 'Over the last several months, we to...
2    {'input': 'work came from optimizing the conne...
3    {'input': 'of the Amazon shopping experience f...
4    {'input': 'Beyond geographic expansion, we’ve ...
5    {'input': 'month flat fee, enables Prime membe...
6    {'input': 'developer productivity by generatin...
7    {'input': '1997 LETTER TO SHAREHOLDERS
(Reprin...
8    {'input': '• We will make bold rather than tim...
9    {'input': 'Infrastructure
During 1997, we work...
Name: json_element, dtype: object

Now we will apply json loads function on each row of the ‘json_element’ column. ‘json.loads’ is a decoder function in python which is used to decode a json object into a dictionary. ‘apply’ is a popular function in pandas that takes any function and applies to each row of the pandas dataframe or series.

In [21]:
df_final = pd.json_normalize(df_inter['json_element'].apply(json.loads))

Once decoding is done we will apply the json normalize function to the above result. json normalize will convert any semi-structured json data into a flat table. Here it converts the JSON ‘keys’ to columns and its corresponding values to row elements.

In [22]:
df_final

,input
0,Dear shareholders:\nAs I sit down to write my ...
1,"Over the last several months, we took a deep l..."
2,work came from optimizing the connections betw...
3,of the Amazon shopping experience for more tha...
4,"Beyond geographic expansion, we’ve been workin..."
5,"month flat fee, enables Prime members to get a..."
6,developer productivity by generating code sugg...
7,1997 LETTER TO SHAREHOLDERS\n(Reprinted from t...
8,• We will make bold rather than timid investme...
9,"Infrastructure\nDuring 1997, we worked hard to..."


### Uploading data to S3

Next, we need to upload our training dataset to S3:

In [23]:
s3_location = f"s3://{sagemaker_session_bucket}/bedrock/finetuning/train-continual-pretraining.jsonl"
s3_output = f"s3://{sagemaker_session_bucket}/bedrock/finetuning/output"

In [24]:
!aws s3 cp ./train-continual-pretraining.jsonl $s3_location


The user-provided path data/train-continual-pretraining.jsonl does not exist.


Now we can create the fine-tuning job. 

### ^^ **Note:** Make sure the IAM role you're using has these [IAM policies](https://docs.aws.amazon.com/bedrock/latest/userguide/model-customization-iam-role.html) attached that allow Amazon Bedrock access to the specified S3 buckets ^^

## 2. Fine-tuning

In [25]:
import time
timestamp = int(time.time())

In [26]:
job_name = "titan-{}".format(timestamp)
job_name

'titan-1698893278'

In [27]:
custom_model_name = "custom-{}".format(job_name)
custom_model_name

'custom-titan-1698893278'

In [ ]:
bedrock_admin.create_model_customization_job(
    customizationType="CONTINUAL_PRE_TRAINING",  # or FINE_TUNING
    jobName=job_name,
    customModelName=custom_model_name,
    roleArn=role,
    baseModelIdentifier=base_model_id,
    hyperParameters = {
        "epochCount": "10",
        "batchSize": "1",
        "learningRate": "0.000001",
        "learningRateWarmupSteps": "0"
    },
    trainingDataConfig={"s3Uri": s3_location},
    outputDataConfig={"s3Uri": s3_output},
)

In [ ]:
status = bedrock_admin.get_model_customization_job(jobIdentifier=job_name)["status"]
status

# Let's periodically check in on the progress.
### The next cell might run for ~40min

In [ ]:
import time

status = bedrock_admin.get_model_customization_job(jobIdentifier=job_name)["status"]

while status == "InProgress":
    print(status)
    time.sleep(30)
    status = bedrock_admin.get_model_customization_job(jobIdentifier=job_name)["status"]
    
print(status)

In [ ]:
completed_job = bedrock_admin.get_model_customization_job(jobIdentifier=job_name)
completed_job

## 3. Testing

Now we can test the fine-tuned model

In [ ]:
bedrock_admin.list_custom_models()

In [ ]:
for job in bedrock_admin.list_model_customization_jobs()["modelCustomizationJobSummaries"]:
    print("-----\n" + "jobArn: " + job["jobArn"] + "\njobName: " + job["jobName"] + "\nstatus: " + job["status"] + "\ncustomModelName: " + job["customModelName"])

## GetCustomModel

In [ ]:
bedrock_admin.get_custom_model(modelIdentifier=custom_model_name)

In [ ]:
custom_model_arn = bedrock_admin.get_custom_model(modelIdentifier=custom_model_name)['modelArn']
custom_model_arn

In [ ]:
base_model_arn = bedrock_admin.get_custom_model(modelIdentifier=custom_model_name)['baseModelArn']
base_model_arn

## **Note:** To invoke custom models, you need to first create a provisioned throughput resource and make requests using that resource.

In [ ]:
provisioned_model_name = "{}-provisioned".format(custom_model_name)
provisioned_model_name

## !! **Note:** SDK currently only supports 1 month and 6 months commitment terms. Go to Bedrock console to manually purchase no commitment term option for testing !!

In [ ]:
# bedrock_admin.create_provisioned_model_throughput(
#     modelUnits = 1,
#     commitmentDuration = "OneMonth", ## Note: SDK is currently missing No Commitment option
#     provisionedModelName = provisioned_model_name,
#     modelId = base_model_arn
# ) 

## ListProvisionedModelThroughputs

In [ ]:
bedrock_admin.list_provisioned_model_throughputs()["provisionedModelSummaries"]

## GetProvisionedModelThroughput

In [ ]:
# provisioned_model_name = "<YOUR_PROVISIONED_MODEL_NAME>" # e.g. custom-titan-1698257909-provisioned

In [ ]:
provisioned_model_arn = bedrock_admin.get_provisioned_model_throughput(
     provisionedModelId=provisioned_model_name)["provisionedModelArn"]
provisioned_model_arn

In [ ]:
deployment_status = bedrock_admin.get_provisioned_model_throughput(
    provisionedModelId=provisioned_model_name)["status"]
deployment_status

## The next cell might run for ~10min

In [ ]:
import time

deployment_status = bedrock_admin.get_provisioned_model_throughput(
    provisionedModelId=provisioned_model_name)["status"]

while deployment_status == "Creating":
    
    print(deployment_status)
    time.sleep(30)
    deployment_status = bedrock_admin.get_provisioned_model_throughput(
        provisionedModelId=provisioned_model_name)["status"]  
    
print(deployment_status)

# Qualitative Results with Zero Shot Inference AFTER Fine-Tuning

As with many GenAI applications, a qualitative approach where you ask yourself the question "is my model behaving the way it is supposed to?" is usually a good starting point. In the example below (the same one we started this notebook with), you can see how the fine-tuned model is able to create a reasonable summary of the dialogue compared to the original inability to understand what is being asked of the model.

In [ ]:
response = bedrock_runtime.invoke_model(
    # modelId needs to be Provisioned Throughput Model ARN
    modelId=provisioned_model_arn,
    body="""
{
  "inputText": "What is the size of the consumer business in 2022?",
  "textGenerationConfig":{
    "maxTokenCount": 50, 
    "stopSequences": [],
    "temperature": 0.1,
    "topP": 0.9
  }
}
"""
)

response_body = response["body"].read().decode('utf8')
print(response_body)

print(json.loads(response_body)["results"][0]["outputText"])

In [ ]:
# response = bedrock_runtime.invoke_model(
#     # modelId needs to be Provisioned Throughput Model ARN
#     modelId=provisioned_model_arn,
#     body="""
# {"inputText": "Who are the authors of 'Generative AI on AWS'?",
#  "textGenerationConfig":{
#   "maxTokenCount":1000,
#   "stopSequences":[],
#   "temperature":1,
#   "topP":0.9
#  }
# }
# """)

# response_body = response["body"].read().decode('utf8')
# print(response_body)

# print(json.loads(response_body)["results"][0]["outputText"])

## Delete Provisioned Throughput

When you're done testing, you can delete Provisioned Throughput to stop charges

In [ ]:
# bedrock_admin.delete_provisioned_model_throughput(
#     provisionedModelId = provisioned_model_name
# )